In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

In [3]:
gdp_fpath = "../data01_Raw/GDP/Agriculture_GDP_World_Bank.xlsx"

In [4]:
gdp_df = pd.read_excel(gdp_fpath)

In [5]:
gdp_df

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,..,...,24.390874,22.810663,22.137041,20.634323,25.740314,26.420199,22.042897,25.773971,26.8206,..
1,Africa Eastern and Southern,AFE,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,..,...,10.809624,11.139972,11.91545,12.762394,12.847244,12.545027,11.987675,12.502325,14.665698,..
2,Africa Western and Central,AFW,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,..,...,21.080401,20.126763,19.652742,20.590106,21.006218,20.805289,20.931406,21.257161,23.004831,..
3,Albania,ALB,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,..,...,18.766802,19.565158,19.990181,19.780214,19.849994,19.022149,18.429479,18.391244,19.121963,..
4,Algeria,DZA,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,..,...,8.770686,9.851117,10.286397,11.578707,12.220095,11.755903,11.874201,12.336212,14.134793,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Find the first entry for all countries
first_data = {}
for row_index, row in gdp_df.iterrows():
    # I know this is brute force but... oh well
    country_name = row['Country Name']
    #if country_name != "Burkina Faso":
    #    continue
    for cur_yr in range(1960, 2022):
        col_str = f"{cur_yr} [YR{cur_yr}]"
        row_val = row[col_str]
        #print(f"{country_name} {col_str} = {row_val}")
        if row_val != "..":
            # We found the first non-missing entry
            first_data[country_name] = (cur_yr, row_val)
            break

In [7]:
first_df = pd.DataFrame.from_dict(first_data, orient="index")

In [8]:
first_df.columns = ['year', 'ag_pct']

In [9]:
first_df.sort_values(by='ag_pct', ascending=False)

,year,ag_pct
Lesotho,1960,88.184615
Liberia,2000,76.074542
Niger,1960,75.324070
Rwanda,1965,74.791704
Oman,1961,74.233129
...,...,...
"Hong Kong SAR, China",2000,0.087327
San Marino,2015,0.020054
NaN,1960,NaN
Data from database: World Development Indicators,1960,NaN


In [11]:
ranked_fpath = "../data01_Raw/GDP/GDP_first_ranked.csv"
first_df.to_csv(ranked_fpath)
first_df.to_pickle(ranked_fpath.replace(".csv",".pkl"))

In [27]:
def get_country_data(df, country_name):
    print(country_name)
    # Keep just one country
    country_df = gdp_df[gdp_df['Country Name'] == country_name].copy().melt()
    # Transform to long format
    country_df = country_df[country_df['value'] != ".."].copy()
    # Keep just year and value rows
    country_df = country_df.loc[country_df['variable'].str.startswith('19') | country_df['variable'].str.startswith('20')].copy()
    first_row = country_df.iloc[0]
    last_row = country_df.iloc[-1]
    first_yr = first_row['variable'].split()[0]
    last_yr = last_row['variable'].split()[0]
    first_ag = first_row['value']
    last_ag = last_row['value']
    country_data = {
        'name': country_name,
        'df': country_df,
        'first_year': first_yr,
        'first_ag': first_ag,
        'last_year': last_yr,
        'last_ag': last_ag,
    }
    return country_data

In [28]:
data_map = {langcode: get_country_data(gdp_df, country) for langcode, country in country_map.items()}

Ethiopia
Germany
Greece
United States
Cuba
Iran, Islamic Rep.
Hungary
Armenia
Italy
Georgia
Mongolia
Romania
Russian Federation
Vietnam
China


In [29]:
ag_estimates = {langcode: (data['first_year'],data['first_ag']) for langcode, data in data_map.items()}

In [30]:
ag_estimates

{'am': ('1981', 54.73913635210796),
 'de': ('1991', 1.101526043637281),
 'el': ('1995', 7.375806358492404),
 'en': ('1997', 1.3369117103816948),
 'es': ('1970', 12.121357275415804),
 'fa': ('1960', 25.900601532239886),
 'hu': ('1995', 7.178038555068181),
 'hy': ('2012', 17.912681952639666),
 'it': ('1990', 3.182368676759932),
 'ka': ('1980', 23.008849557522122),
 'mn': ('1981', 16.742788220643867),
 'ro': ('1990', 21.809068656020518),
 'ru': ('1989', 15.602094240837697),
 'vi': ('1985', 40.17093765286004),
 'zh': ('1960', 23.175294197673626)}

In [14]:
# Now convert to df
ag_df = pd.DataFrame.from_dict(ag_estimates, orient='index')

In [15]:
ag_df.columns = ["year","ag_pct"]

In [16]:
gdp_path = "./gdp_stats/"
if not os.path.isdir(gdp_path):
    os.mkdir(gdp_path)

In [17]:
ag_df.to_pickle("./gdp_stats/ag_pct.pkl")

In [18]:
data_map['vi']['first_year']

'1985'

In [19]:
data_map['vi']['first_ag']

40.17093765286004

In [20]:
data_map

{'am': {'name': 'Ethiopia',
  'df':          variable      value
  25  1981 [YR1981]  54.739136
  26  1982 [YR1982]   53.25651
  27  1983 [YR1983]  54.377092
  28  1984 [YR1984]  47.898258
  29  1985 [YR1985]  52.578448
  30  1986 [YR1986]  50.986366
  31  1987 [YR1987]   49.12477
  32  1988 [YR1988]  48.593054
  33  1989 [YR1989]  48.390914
  34  1990 [YR1990]  49.491248
  35  1991 [YR1991]  58.666062
  36  1992 [YR1992]  63.831336
  37  1993 [YR1993]  59.945298
  38  1994 [YR1994]  52.699451
  39  1995 [YR1995]  51.924704
  40  1996 [YR1996]  51.165368
  41  1997 [YR1997]  54.028989
  42  1998 [YR1998]  49.061908
  43  1999 [YR1999]  45.485463
  44  2000 [YR2000]  44.666018
  45  2001 [YR2001]  42.340571
  46  2002 [YR2002]  38.692372
  47  2003 [YR2003]  37.285055
  48  2004 [YR2004]  38.679755
  49  2005 [YR2005]  41.174499
  50  2006 [YR2006]   42.52417
  51  2007 [YR2007]  42.265012
  52  2008 [YR2008]  45.184558
  53  2009 [YR2009]  45.882694
  54  2010 [YR2010]   41.44682
  55 